<a href="https://colab.research.google.com/github/harim061/KoBERT/blob/main/kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-2a8b4npj/kobert-tokenizer_571d5a8583ca4b2e9a744e658221e7b3
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-2a8b4npj/kobert-tokenizer_571d5a8583ca4b2e9a744e658221e7b3
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [4]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# GPU 사용 시
device = torch.device("cuda:0")

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [8]:
import pandas as pd
data = pd.read_csv('dataset.csv',encoding="cp949")

In [9]:
data.loc[(data['감정_소분류'] == "두려운") | (data['감정_소분류'] == "마비된"), '감정_소분류'] = 0
data.loc[(data['감정_소분류'] == "외로운") | (data['감정_소분류'] == "우울한") | (data['감정_소분류'] == "가난한, 불우한") | (data['감정_소분류'] == "슬픔") | (data['감정_소분류'] == "한심한") | (data['감정_소분류'] == "고립된")| (data['감정_소분류'] == "염세적인") |(data['감정_소분류'] == "희생된")|(data['감정_소분류'] == "비통한")|(data['감정_소분류'] == "상처"), '감정_소분류'] = 1
data.loc[(data['감정_소분류'] == "신뢰하는") |(data['감정_소분류'] == "자신하는"), '감정_소분류'] = 2
data.loc[(data['감정_소분류'] == "남의 시선을 의식하는") | (data['감정_소분류']=='초조한')| (data['감정_소분류']=='걱정스러운')| (data['감정_소분류']=='부끄러운'), '감정_소분류'] = 3
data.loc[(data['감정_소분류'] == "구역질 나는") | (data['감정_소분류'] == "악의적인") | (data['감정_소분류'] == "열등감") | (data['감정_소분류'] == "분노") | (data['감정_소분류']=="혐오스러운") | (data['감정_소분류']=="노여워하는"), '감정_소분류'] = 4
data.loc[(data['감정_소분류'] == "낙담한") | (data['감정_소분류'] == "눈물이 나는") | (data['감정_소분류'] == "좌절한") | (data['감정_소분류'] == "충격 받은") | (data['감정_소분류'] == "버려진") | (data['감정_소분류'] == "실망한")| (data['감정_소분류'] == "괴로워하는")| (data['감정_소분류'] == "배신당한"), '감정_소분류'] = 5
data.loc[data['감정_소분류'] == "억울한", '감정_소분류'] = 6
data.loc[(data['감정_소분류'] == "안달하는") | (data['감정_소분류']=="불안"), '감정_소분류'] = 7
data.loc[data['감정_소분류'] == "감사하는", '감정_소분류'] = 8
data.loc[(data['감정_소분류'] == "방어적인") |(data['감정_소분류'] == "조심스러운")|(data['감정_소분류'] == "취약한")|(data['감정_소분류'] == "회의적인"), '감정_소분류'] = 9
data.loc[(data['감정_소분류'] == "당황") | (data['감정_소분류'] == "당혹스러운")| (data['감정_소분류'] == "혼란스러운"), '감정_소분류'] = 10
data.loc[(data['감정_소분류'] == "편안한") | (data['감정_소분류'] == "느긋")| (data['감정_소분류'] == "안도"), '감정_소분류'] = 11
data.loc[(data['감정_소분류'] == "툴툴대는") | (data['감정_소분류'] == "짜증내는") | (data['감정_소분류'] == "스트레스 받는")| (data['감정_소분류'] == "질투하는")| (data['감정_소분류'] == "환멸을 느끼는")| (data['감정_소분류'] == "성가신"), '감정_소분류'] = 12
data.loc[data['감정_소분류'] == "후회되는", '감정_소분류'] = 13
data.loc[data['감정_소분류'] == "만족스러운", '감정_소분류'] = 14
data.loc[data['감정_소분류'] == "죄책감의", '감정_소분류'] = 15
data.loc[(data['감정_소분류']=='신이 난') | (data['감정_소분류']=='흥분')| (data['감정_소분류']=='기쁨'),'감정_소분류']= 16


In [10]:
data_list = []
for ques, label in zip(data['사람문장1'], data['감정_소분류'])  :
    data = []
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [11]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')


In [12]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [13]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [20]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [21]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tokenizer, vocab,  max_len, True, False)

In [22]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [23]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=17,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [24]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

# train

In [25]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-25-0cd8b201d8af>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/83 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.836052656173706 train acc 0.03125
epoch 1 train acc 0.27503765060240964


<ipython-input-25-0cd8b201d8af>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/21 [00:00<?, ?it/s]

...

epoch 9 test acc 0.5118743926141885


  0%|          | 0/83 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.44765564799308777 train acc 0.875
epoch 10 train acc 0.9182981927710844


  0%|          | 0/21 [00:00<?, ?it/s]

epoch 10 test acc 0.5155946307094267


In [32]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("두렵+마비")
            elif np.argmax(logits) == 1:
                test_eval.append("외롭 우울")
            elif np.argmax(logits) == 2:
                test_eval.append("자신만만")
            elif np.argmax(logits) == 3:
                test_eval.append("초조")
            elif np.argmax(logits) == 4:
                test_eval.append("우엑")
            elif np.argmax(logits) == 5:
                test_eval.append("낙담")
            elif np.argmax(logits) == 6:
                test_eval.append("억울")
            elif np.argmax(logits) == 7:
                test_eval.append("안달")
            elif np.argmax(logits) == 8:
              test_eval.append("ㄳ")
            elif np.argmax(logits) == 9:
                test_eval.append("방어")
            elif np.argmax(logits) == 10:
              test_eval.append("당황")
            elif np.argmax(logits) == 11:
                test_eval.append("편안")
            elif np.argmax(logits) == 12:
                test_eval.append("짜증")
            elif np.argmax(logits) == 13:
                test_eval.append("후회")
            elif np.argmax(logits) == 14:
                test_eval.append("만족")
            elif np.argmax(logits) == 15:
                test_eval.append("죄책")
            elif np.argmax(logits) == 16:
                test_eval.append("기뻐 흥분")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [33]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 0


# 자동화시키기

In [34]:
data_result = pd.read_csv('댓글.csv',encoding='cp949')


In [35]:
dataR = data_result.loc[:,'댓글']


In [36]:
for i in dataR:
    print(i+" : ",end='')
    predict(i)
    print('\n')

몬베베되고 첫인상이랑 젤 다른 멤버, 툭툭나오는 애교들이 진짜 너무 귀여움. 우리 고앵이 : 

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 내용에서 짜증 느껴집니다.


04:50 창균이 웃는 거 개귀엽다 : >> 입력하신 내용에서 기뻐 흥분 느껴집니다.


몽베베 눈에는 그저 깜찍 겸둥 큐티 아기 막내인데 : >> 입력하신 내용에서 편안 느껴집니다.


몬엑 생기기전에 리얼리티 노머시부터 봤던 사람인데 진심 창균님 정말 진중하고 착하고 대단한 사람임,, 랩도 잘하고 목소리도 좋고 잘생기고 랩도 잘함 ㅠㅠ : >> 입력하신 내용에서 자신만만 느껴집니다.


처음보는 사람들에게는 막내같지 않겠지만 형들과 몬베베에게는 엄청난 사랑과 우쭈쭈를 받는 활동명을 잃어버린 균이 : >> 입력하신 내용에서 낙담 느껴집니다.


1:07 이와중에 허니 진짜 꾸꾸까까 몇 천 번째 보는데 최강 귀여움.....뭐냐 이주헌 진짜 겁나 사랑한다???? : >> 입력하신 내용에서 자신만만 느껴집니다.


아 몬엑분들 왤케 귀여우세요...?; 애교가 아주 몸에 베여있네ㅠ 아 진짜 사람 보쌈해가고싶게? : >> 입력하신 내용에서 짜증 느껴집니다.


이게 머선일이고 우리 창균이 분량 갱장히 조으다ㅠㅠㅠㅠ : >> 입력하신 내용에서 우엑 느껴집니다.


Im Changkyun: Producer, lyricist, rapper, vocalist, dancer, model and forever our beloved maknae. Please release your new album soon!! : >> 입력하신 내용에서 두렵+마비 느껴집니다.


9:01 너무 귀여워서 심장이 멎을뻔…ㅎ하ㅏㅠ : >> 입력하신 내용에서 기뻐 흥분 느껴집니다.


독보적인 시크남 아엠 ?? 독보적 케릭터 귀욤과 터프함도 겸비한 최애 연예인  주허니 : >> 입력하신 내용에서 짜증 느껴집니다.


Hahahah that legendary clip of jooheon singing and changkyun just being shocked of his energy  : >> 입력하신 내용에서 자신만만 느껴집니다.


우리 짱규니 말예쁘고 다